In [0]:
# Set configuration settings
config = {
    "spark.cosmos.accountEndpoint": "https://it-step-cosmos-db2.documents.azure.com:443/",
    "spark.cosmos.accountKey": "E9ziM0JVhKPeEDNoM0KYzTePmhs7cJGYn0ih28snADCfeCqz5qTLHSlwPU83qQVHweDVTMB5CMIdACDbxkzaYA==",
    "spark.cosmos.database": "cosmicworks",
    "spark.cosmos.container": "products"
}


In [0]:
# Configure Catalog Api    
spark.conf.set("spark.sql.catalog.cosmosCatalog", "com.azure.cosmos.spark.CosmosCatalog")
spark.conf.set("spark.sql.catalog.cosmosCatalog.spark.cosmos.accountEndpoint", config["spark.cosmos.accountEndpoint"])
spark.conf.set("spark.sql.catalog.cosmosCatalog.spark.cosmos.accountKey", config["spark.cosmos.accountKey"])


In [0]:
# Create a database using the Catalog API
spark.sql("CREATE DATABASE IF NOT EXISTS cosmosCatalog.cosmicworks;")


DataFrame[]

In [0]:
# Create a products container using the Catalog API
spark.sql("CREATE TABLE IF NOT EXISTS cosmosCatalog.cosmicworks.products USING cosmos.oltp TBLPROPERTIES(partitionKeyPath = '/category', autoScaleMaxThroughput = '1000')")


DataFrame[]

In [0]:
# Create an employees container using the Catalog API
spark.sql("CREATE TABLE IF NOT EXISTS cosmosCatalog.cosmicworks.employees USING cosmos.oltp TBLPROPERTIES(partitionKeyPath = '/organization,/department,/team', manualThroughput = '400')")


DataFrame[]

In [0]:
# Create sample data
products = (
    ("68719518391", "gear-surf-surfboards", "Yamba Surfboard", 12, 850.00, False),
    ("68719518371", "gear-surf-surfboards", "Kiama Classic Surfboard", 25, 790.00, True)
)
